<a href="https://colab.research.google.com/github/wrn6494/8-Week-SQL-Challenge/blob/main/Case_Study_7_Balanced_Tree_Clothing_Co_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **8 Week SQL Challenge : Case Study #7 - Balanced Tree Clothing Co.**

Full case study available [here.](https://8weeksqlchallenge.com/case-study-7/)

**Probem Statement/Business Task**

Balanced Tree Clothing Company prides themselves on providing an optimised range of clothing and lifestyle wear for the modern adventurer.

The company wants assistance in analysing their sales performance and in generating a basic financial report.

# **Case Study Questions**

1. **High Level Sales Analysis**
  1.   What was the total quantity sold for all products?
  2.   What is the total generated revenue for all products before discounts?
  3.   What was the total discount amount for all products?


2. **Transaction Analysis**
  1.   How many unique transactions were there?
  2.   What is the average unique products purchased in each transaction?
  3.  What are the 25th, 50th and 75th percentile values for the revenue per transaction?
  4.   What is the average discount value per transaction?
  5.   What is the percentage split of all transactions for members vs non-members?
  6.   What is the average revenue for member transactions and non-member transactions?

3. **Product Analysis**
  1.  What are the top 3 products by total revenue before discount?
  2.  What is the total quantity, revenue and discount for each segment?
  3.  What is the top selling product for each segment?
  4.  What is the total quantity, revenue and discount for each category?
  5.  What is the top selling product for each category?
  6.  What is the percentage split of revenue by product for each segment?
  7.  What is the percentage split of revenue by segment for each category?
  8.  What is the percentage split of total revenue by category?
  9.  What is the total transaction “penetration” for each product? (hint: penetration = number of transactions where at least 1 quantity of a product was purchased divided by total number of transactions)
  10.  What is the most common combination of at least 1 quantity of any 3 products in a 1 single transaction?

4. **Reporting Challenge**

Write a single SQL script that combines all of the previous questions into a scheduled report that the Balanced Tree team can run at the beginning of each month to calculate the previous month’s values.

Imagine that the Chief Financial Officer (which is also Danny) has asked for all of these questions at the end of every month.

He first wants you to generate the data for January only - but then he also wants you to demonstrate that you can easily run the samne analysis for February without many changes (if at all).

Feel free to split up your final outputs into as many tables as you need - but be sure to explicitly reference which table outputs relate to which question for full marks :)

5. **Bonus Challenge**

Use a single SQL query to transform the product_hierarchy and product_prices datasets to the product_details table.

Hint: you may want to consider using a recursive CTE to solve this problem!



# **1. **High Level Sales Analysis****

  1.   What was the total quantity sold for all products?


    SELECT DISTINCT product_name, SUM(qty)
    FROM balanced_tree.product_details d
    JOIN balanced_tree.sales s
    ON d.product_id=s.prod_id
    GROUP BY product_name

   2.   What is the total generated revenue for all products before discounts?


    SELECT SUM(price*qty)
    FROM balanced_tree.sales

| sum |
| -----------| 
|   1289453    | 



   3.   What was the total discount amount for all products?


    WITH A AS
    (
    SELECT DISTINCT txn_id,discount
    FROM balanced_tree.sales
    )

    SELECT SUM(discount)
    FROM A


| sum |
| -----------| 
|  30226    | 
    

# **2. **Transaction Analysis****

  1.   How many unique transactions were there?


    SELECT COUNT(DISTINCT txn_id)
    FROM balanced_tree.sales

| sum |
| ----| 
| 2500| 

2.   What is the average unique products purchased in each transaction?


    WITH A AS
    (
    SELECT COUNT(DISTINCT prod_id)
    FROM balanced_tree.sales
    GROUP BY txn_id
    )

    SELECT ROUND(AVG(count)) AS average_products
    FROM A

| average_products |
| ----| 
| 6| 

3.  What are the 25th, 50th and 75th percentile values for the revenue per transaction?



    WITH A AS
    (
    SELECT DISTINCT txn_id,discount
    FROM balanced_tree.sales
    ),
    B AS
    (
    SELECT DISTINCT txn_id,SUM(price*qty) AS profit
    FROM balanced_tree.sales
    GROUP BY txn_id
    ),
    C AS
    (
    SELECT (profit-A.discount) AS revenue
    FROM A
    JOIN B
    USING (txn_id)
    )

    SELECT 
      PERCENTILE_DISC(0.25) WITHIN GROUP (ORDER BY revenue ASC) AS percentile_25,
        PERCENTILE_DISC(0.5) WITHIN GROUP (ORDER BY revenue ASC) AS percentile_50,
        PERCENTILE_DISC(0.75) WITHIN GROUP (ORDER BY revenue ASC) AS percentile_75
    FROM C


| percentile_25 | 	percentile_50 |percentile_75|
| ----| ----|  ----|  
| 364| 497| 636|


4.   What is the average discount value per transaction?


    WITH A AS
    (
    SELECT DISTINCT txn_id,discount
    FROM balanced_tree.sales
    )

    SELECT ROUND(AVG(discount)) AS average_discount
    FROM A

| average__discount |
| ----| 
| 12| 

5.   What is the percentage split of all transactions for members vs non-members?


    WITH A AS
    (
    SELECT DISTINCT txn_id,member
    FROM balanced_tree.sales
    )

    SELECT ((SUM(CASE WHEN member='t' THEN 1 ELSE 0 END))*100)/COUNT(txn_id) AS member_perc
    FROM A


| member_perc |
| ----| 
| 60| 

Members: 60%, Non-members: 40%

6.   What is the average revenue for member transactions and non-member transactions?


    WITH A AS
    (
    SELECT DISTINCT txn_id,discount
    FROM balanced_tree.sales
    ),
    B AS
    (
    SELECT DISTINCT txn_id,SUM(price*qty) AS profit
    FROM balanced_tree.sales
    GROUP BY txn_id
    )

    SELECT ROUND(AVG(profit-A.discount),2) AS member_revenue
    FROM A
    JOIN B
    USING (txn_id)
    JOIN balanced_tree.sales
    USING (txn_id)
    WHERE member='t'

| member_revenue |
| ----| 
| 544.77| 

    SELECT ROUND(AVG(profit-A.discount),2) AS member_revenue
    FROM A
    JOIN B
    USING (txn_id)
    JOIN balanced_tree.sales
    USING (txn_id)
    WHERE member='f'

| member_revenue |
| ----| 
| 546.52| 

# **3. **Product Analysis****

1.  What are the top 3 products by total revenue before discount?


    SELECT product_name,SUM(s.price*qty)
    FROM balanced_tree.sales s
    JOIN balanced_tree.product_details d
    ON s.prod_id=d.product_id
    GROUP BY product_name
    ORDER BY product_name DESC
    LIMIT 3

| product_name |	sum |
| ----| ----|  
|White Tee Shirt - Mens |152000|  
|White Striped Socks - Mens |62135|  
|Teal Button Up Shirt - Mens |36460|  

2.  What is the total quantity, revenue and discount for each segment?


    SELECT segment_name,
      SUM(qty) AS qty,
      SUM(s.price*qty) AS revenue,
      SUM(discount) AS discount
    FROM balanced_tree.sales s
    JOIN balanced_tree.product_details d
    ON s.prod_id=d.product_id
    GROUP BY segment_name
    ORDER BY segment_name DESC


| segment_name |	qty |	revenue |	discount |
| ----| ----| ----| ----|
|Socks|11217|307977 |45465| 
|Shirt|11265| 406143|46043| 
|Jeans|11349|208350 |45740|
|Jacket|11385| 366983|45452|

3.  What is the top selling product for each segment?


    SELECT segment_name,product_name,
    SUM(qty) AS qty,
    SUM(s.price*qty) AS revenue,
    SUM(discount) AS discount
    FROM balanced_tree.sales s
    JOIN balanced_tree.product_details d
    ON s.prod_id=d.product_id
    GROUP BY segment_name,product_name
    ORDER BY segment_name,revenue DESC, qty DESC

4.  What is the total quantity, revenue and discount for each category?


    SELECT category_name,
      SUM(qty) AS qty,
      SUM(s.price*qty) AS revenue,
      SUM(discount) AS discount
    FROM balanced_tree.sales s
    JOIN balanced_tree.product_details d
    ON s.prod_id=d.product_id
    GROUP BY category_name

| category_name |	qty |	revenue |	discount |
| ----| ----| ----| ----|
|Mens|22482|714120|91508| 
|Womens|22734| 575333|91192| 

5.  What is the top selling product for each category?


    SELECT category_name,product_name,
    SUM(qty) AS qty,
    SUM(s.price*qty) AS revenue,
    SUM(discount) AS discount
    FROM balanced_tree.sales s
    JOIN balanced_tree.product_details d
    ON s.prod_id=d.product_id
    GROUP BY category_name,product_name
    ORDER BY category_name,revenue DESC, qty DESC

6.  What is the percentage split of revenue by product for each segment?


    WITH A AS 
    (      
    SELECT segment_name,product_name,
    SUM(s.price*qty) AS revenue
    FROM balanced_tree.sales s
    JOIN balanced_tree.product_details d
    ON s.prod_id=d.product_id
    GROUP BY segment_name,product_name
    ),
    B AS
    (
    SELECT segment_name,
    SUM(s.price*qty) AS total_segment_revenue
    FROM balanced_tree.sales s
    JOIN balanced_tree.product_details d
    ON s.prod_id=d.product_id
    GROUP BY segment_name
    )

    SELECT segment_name, product_name,((revenue*100)/total_segment_revenue) AS percentage
    FROM A 
    RIGHT JOIN B
    USING (segment_name)
    ORDER BY segment_name

7.  What is the percentage split of revenue by segment for each category?


    WITH A AS 
    (      
    SELECT category_name,segment_name,
    SUM(s.price*qty) AS revenue
    FROM balanced_tree.sales s
    JOIN balanced_tree.product_details d
    ON s.prod_id=d.product_id
    GROUP BY category_name,segment_name
    ),
    B ASa
    (
    SELECT category_name,
    SUM(s.price*qty) AS total_category_revenue
    FROM balanced_tree.sales s
    JOIN balanced_tree.product_details d
    ON s.prod_id=d.product_id
    GROUP BY category_name
    )

    SELECT category_name, segment_name,((revenue*100)/total_category_revenue) AS percentage
    FROM A 
    RIGHT JOIN B
    USING (category_name)
    ORDER BY category_name
  


8.  What is the percentage split of total revenue by category?


    WITH A AS
    (
      SELECT category_name,
          SUM(s.price*qty) AS revenue
        FROM balanced_tree.sales s
        JOIN balanced_tree.product_details d
        ON s.prod_id=d.product_id
        GROUP BY category_name
    ),
    B AS
    (
        SELECT
          SUM(s.price*qty) AS total_revenue
        FROM balanced_tree.sales s
    )
    SELECT category_name, ((revenue*100)/ (SELECT total_revenue FROM B)) AS percentage
    FROM A

9.  What is the total transaction “penetration” for each product? (hint: penetration = number of transactions where at least 1 quantity of a product was purchased divided by total number of transactions)

10.  What is the most common combination of at least 1 quantity of any 3 products in a 1 single transaction?

# **4. **Reporting Challenge****

# **5. **Bonus Challenge****